# PRONOUNCEABILITY PROBABILITY: HAVE MAX GRADE
### By: Ethan Ringel, Jayden Andrews, Kai Walter

Our group shares a burning interest in the English language, more specifically, how words are created and spoken. Our thoughts led us to want to perform Data Science on word pronounceability, more specifically on how to determine whether a word is pronounceable or not. We attempted to develop a heuristic which, when given a completely random string of letters, could determine whether that word is pronounceable by English speaking people. 

Additionally, after developing this heuristic, we decided to test its effectiveness by comparing the accuracy of our heuristic to the accuracy of various machine learning algorithms which we would run on datasets of pronounceable and unpronounceable words.

# Task 1: Data Collection/Curation + Parsing

In order to run ML algorithms to determine whether a word is pronounceable or not as well as run our self made heuristic, we needed a dataset containing words which we could identify as either pronounceable or not pronounceable*.

*One thing to note about the pronounceability of a word is that there exists some nuance to this issue. Technically speaking, an English speaker may attempt to pronounce any given sequence of letters and deem a particular pronunciation correct, which would then make that word “pronounceable.” However, our classification of pronounceability focuses more on how easy it would be for an English speaker to view a word and determine the correct pronunciation. For example, if we took two gibberish words “matcutious” and “xzvuopowmf,” we see that one of these words seems far more sensicle given prior knowledge of English conventions. Thus we would likely deem “matcutious” pronounceable and “xzvuopowmf” unpronounceable.

### Idea #1:
We imported a large (literal) dictionary containing upwards of 300,000 English words; however, we ran into two primary issues with this approach:
Regardless of which dataset we imported, the dataset would contain many nonsensical words as well as words which contained many difficult characters to handle such as spaces, hyphens, apostrophes, etc. and there was little we could do to remove these words or work around them
With over 300,000 data points, attempting to represent our data through one-hot encoded bigrams was impossible and simply wouldn’t run on any of our systems

### Idea #2:
Through some brainstorming, we concluded that we could instead import a dataset which contained the most common words in the English language sorted by their general frequency and truncate this dataset at a certain quantity. Our reasoning behind this decision was that we would simultaneously be able to reduce the size of our dataset while filtering out many of the nonsensical words (because they would likely have very low frequency values). Additionally, we reasoned that English words that appear most often in everyday use would be far easier to pronounce for the given English speaker, so a set of the most common words would be more valuable than a set simply containing every word ever created.


### Finalized Approach
With that in mind, we went with idea #2, importing a dataset of the most commonly used English words and isolating the 3000 most common English words, solving our issue of obtaining data for pronounceable words.

Obtaining data for unpronounceable words was an entirely different process. There simply do not exist datasets filled with gibberish that can’t be pronounced, so we decided to write our own method to generate these unpronounceable words. 

In order to generate unpronounceable words, we used three general rules which we viewed as making a word difficult to pronounce.
The word has a long substring of adjacent consonants
The frequency of vowels is relatively low
The word contains the letter q without being followed by u or being at the end of the word

From here we used these rules to generate random strings, determine if they fit any of the unpronounceable criteria, and if so, add them to our collection of unpronounceable words. This then solved our issue of attaining a list of unpronounceable words**.

**There are obviously countless more rules regarding how letters interact in English which relate to pronounceability or lack thereof which we can’t account for in our generation of unpronounceable words. However, our three rules seem to produce rather consistent results in the sense that most of the words generated are quite difficult to pronounce which we deemed sufficient for the sake of our model.

## Task 2: Data management/representation
At this point, we now had a full list of words, both pronounceable and unpronounceable, resulting from our collection of the most common English words and generation of unpronounceable words. From here we needed a way to turn this categorical data into numerical data which could be fed into our various ML algorithms.

For this we decided to one-hot encode bigrams. The point of using bigrams as opposed to another possible means of dividing each word was that bigrams best mimic the syllables contained within each word, which are the base units of each word that determine its pronunciation.

To one-hot encode the bigrams within each word in our collection, we would represent each word with a vector of length 676 (with each element of this vector representing a possible bigram, 262 = 676). Thus our feature vector would be formatted as [“aa”: int, “ab”: int, …, “zz”: int]. For example, if we took the word “abba” our resulting vector would be [“aa”: 0, “ab”: 1, …, “ba”: 1, “bb”: 1, … “zz”: 0].

To isolate the bigram, we defined a helper method which would take a word as an argument and return a list of all the bigrams contained within that word. For example, if we passed in “hello”, the method would return [“he”, “el”, “ll”, “lo”].

Combining this helper method with our intended representation of the data, we could now take our collection of words, isolate the bigrams of each word, and then put our data into one large dataframe, where each word would be represented by a vector of length 676.

### Helper Methods
update_progress: Implements progess bar (added because of time-intensity of certain algorithms)

get_bigrams: As described above, returns bigrams contained within a word

foldl: Small implementation of fold left 

is_probability: Returns whether or not a variable is a valid probability

In [53]:
import time, sys
from IPython.display import clear_output

def update_progress(progress, label: str = ""):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress, 0))

    clear_output(wait = True)
    text = label+"\nProgress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)


def get_bigrams(word:str) -> list:
    return [i+j for i, j in \
            zip(word, word[1:])]


import functools
def foldl(func, xs, acc):
    return functools.reduce(func, xs, acc)

def is_probability(input):
    return isinstance(input, float) and \
        input >= 0 and \
            input <= 1



### Static Variables
letter_likelihood: Holds the likelihood value for possible bigrams based on how often they appear within a dictionary

accuracy_dict: Will contain the accuracy values for each algorithm (heuristic + 5 ML algorithms)

In [38]:
letter_likelihood = {}
accuracy_dict = {}

### Vectorization Code
As described in prose in task 2, these helper methods will help represent a word as a 676-dimensional feature vector where all bigrams are one-hot encoded.

generate_feature_vector: Will generate the feature vector for either a single string or a list of strings

get_vectors_for_series: Generates above feature vectors for a pandas series

In [39]:
from itertools import combinations_with_replacement
import string

def generate_feature_vector(input):
    # we will be outsourcing this to a c subprocess to increase perf
    if("str" in str(type(input))):
        return generate_feature_vector(get_bigrams(input))
    elif("list" in str(type(input))):
        feature_vector = {
            str(bigram) : 1 if (str(bigram[0])+str(bigram[1])) in input else 0 \
                for bigram in \
                    [i+j for i in string.ascii_lowercase for j in string.ascii_lowercase]
        }
        return feature_vector
    else:
        print(input)
        raise TypeError(f"Requires either 'str' or List[str] as input for generate_feature_vector(), found {type(input)}.")
        
import pandas as pd
import numpy as np
def get_vectors_for_series(data: pd.Series, label: str):
    vectors = []
    length = len(data)
    for i in range(length):
        word = data.loc[i]
        if i % 5 == 0 or i == length:
            update_progress(i / length, label=label)
        vectors.append(np.asarray(list(generate_feature_vector(get_bigrams(word)).values())))
    return pd.Series(vectors)

### Dataset Creation Code
These helper methods are used to generate lists of pronounceable and unpronounceable words based on the aforementioned rules regarding generation.

get_n_pronounceable_words: Returns a set of n pronounceable words

get_n_unpronounceable_words: Returns a set of n unpronounceable words

get_dataset: Returns of set of n unpronouceable words and n pronounceable words along with classification labels

In [40]:
import os
import re
import random
import typing
import math

def get_n_pronounceable_words(n: int) -> typing.Set[str]:
    data_path = os.path.normpath(os.path.join(os.path.dirname(os.path.abspath("word_pronounceability.ipynb")), '..', 'unigram_freq.csv'))
    dataframe = pd.read_csv(data_path)
    dataframe = dataframe[dataframe.word.str.len() >= 3]
    dataframe = dataframe.set_axis(range(0, dataframe.shape[0]), axis=0)
    
    sample_size_n = dataframe.sample(n = n)
    return set(sample_size_n["word"])

def get_n_unpronounceable_words(n: int) -> typing.Set[str]:
    def norm(vector:list):
        return math.sqrt(sum([i*i for i in vector]))
        
    words: typing.Set[str] = set()
    while len(words) < n:
        possible_word = ''.join(random.choice(string.ascii_lowercase) for _ in range(random.choice(list(range(3,15)))))

        violates_q_u_rule = "q" in possible_word and "qu" not in possible_word

        num_consecutive_consonants = foldl(lambda x, y: x+1 if y not in list("aeiouy") else 0, possible_word, 0)
        
        contains_no_vowels = num_consecutive_consonants == len(possible_word)

        incorrectness_vector = [0.8*(1 if contains_no_vowels else 0), 0.4*(num_consecutive_consonants/4)]
        incorrectness_vector.append(0.8*(norm(incorrectness_vector)) if violates_q_u_rule else norm(incorrectness_vector))
        # ADJUST IF THE WORDS ARE TOO PRONOUNCEABLE
        if norm(incorrectness_vector) > 0.7 and possible_word not in words:
            words.add(possible_word)
    return words


def get_dataset(size: int) -> pd.DataFrame:
    """Generates a dataset of "words." Half are pronounceable, half are not

    Args:
        size (int): Size of each half of the dataset (pronounceable / unpronounceable)

    Returns:
        pd.DataFrame: dataframe with two columns: "word" and "is_pronounceable" with `2*size` rows.
    """
    data_set = pd.DataFrame(list(get_n_pronounceable_words(size)) + (list((get_n_unpronounceable_words(size)))), columns=["word"])
    data_set["is_pronounceable"] = data_set.index < size
    return data_set


### Heuristic Model
Our initial hypothesis was that the pronounceability of a word correlated very strongly with it's *likelihood*. This is to say, if it is statistically probable that a sequence of letters could make a word, it is also statistically probable that it can be pronounced. This is not without caveats, however: especially because we have idiomatically accepted brand names like "Exxon" which contain strings of letters which no (or at least very few) dictionary words contain. This fact would drive down the likelihood that these such strings of letters would appear, yet we can pronounce them perfectly fine. However, despite these caveats, we feel that this is a reasonable heuristic.

## Heuristic Model Code
pronounceable_score_heuristic: Given a bigram, function will check the number of times this bigram appear within a collection of pronounceable words. Based on the value for each bigram, will then condense these values into a final score for the bigram.

is_pronounceable_heuristic: Given a word, function will disect it into its bigrams; for each bigram within the word it calls the pronouceable_score_heuristic function; finds the average of these scores and returns true (pronounceable) if the value is above a particular threshold and returns false (unpronouceable) if the value if below the threshold.

For the latter, we set the value of the threshold to 0.1 after some tuning. We found that a threshold close to 0 would yield around 50% accuracy and that thesholds above 0.5 resulted in similar accuracies. However, at a value of 0.1, the accuracy was able to spike around 85-90%.

In [42]:
from contextlib import contextmanager
from statistics import mean
def pronounceable_score_heuristic(letters: str) -> float:
    """ Generates a numerical score representing the likelihood that a word is pronounceable.

    Args:
        letters (str): string of length 2 (bigram) containing only alphabetical characters to check our dataset for occurrences of

    Returns:
        float: a score representing the likelihood that we can pronounce this string of letters. 0.5 is generally pronounceable, 0.2 is not.
    """
    assert len(letters) == 2
    # if we have already checked this bigram, it'll be in our letter_likelihood dictionary, we can return it
    if letters in letter_likelihood:
        return letter_likelihood[letters]

    # otherwise,
    # check dataset for occurrences of [letters].
    data = list(get_n_pronounceable_words(9000))
    proportion = dict(in_line=0, not_in=0)
    for word in data:
        proportion['in_line' if letters in word else "not_in"] += 1
    proportion['in_line'] -= 1 if (not proportion['in_line'] > 0) else 0
    # if the set of letters is never found, then it almost certainly can't be pronounced, or possibly is simply not in our dataset.
    # return the amount of times it was found divided by the total lines in the file (multiply by 10 to trim leading zeroes)
    letter_likelihood[letters] = (proportion['in_line'] / sum([proportion[key] for key in proportion]))*10 
    return letter_likelihood[letters]
def is_pronounceable_heuristic(word: str) -> bool:
    # Threshold of 0 gives 50% accuracy, anything above  0.5 gives 50% accuracy
    THRESHOLD = 0.1

    # turns word into list of bigrams into their likelihood of showing up in our dataset
    # "hello" ->  ["he", "el", "ll", "lo"] -> [0.45..., 0.62..., 0.57..., 0.44...]
    average_score = mean([pronounceable_score_heuristic(bigram) for bigram in get_bigrams(word)])

    # if the average pronounceability score is too low, we assume it isn't pronounceable.
    return average_score >= THRESHOLD


# this function allows for more concise and readable code in our test flow.
# uses a contextlib contextmanager to implement __enter__ and __exit__ for our function so we can use it in 'with' statements.
@contextmanager
def heuristic_function():
    function = is_pronounceable_heuristic
    try:
        yield function
    finally:
        pass

Now that we have defined our heuristic model, we can test it on a large segment of data and check to see if it appropriately classifies incoming data. Additionally, now that we have defined our helper methods to organize the necessary data for running our ML algorithms, we can run those and compare the scores to that of our heuristic.

# Task 3: Exploratory data analysis
In order to properly display our data, we decided to use five machine learning algorithms. We decided to use Naive Bayes’ algorithm, a k-Nearest neighbor algorithm, semi-supervised model label propagation, a support vector machine, and a neural network. We recognize that not all of these algorithms were necessarily comparable to each other, but we thought that it was important to compare the differences between all of them to properly display our findings. The results were as follows.

### Heuristic Test

In [43]:
skip_heuristic = False
with heuristic_function() as is_pronounceable:
    if not skip_heuristic:
        test_data = pd.DataFrame(columns=["word", "is_pronounceable"])
        pronounceable_data = pd.DataFrame(columns=["word"], data=pd.Series(list(get_n_pronounceable_words(1000))))
        pronounceable_data["is_pronounceable"] = True

        unpronounceable_data = pd.DataFrame(columns=["word"], data=pd.Series(list(get_n_unpronounceable_words(1000))))
        unpronounceable_data["is_pronounceable"] = False

        test_data = pd.concat(objs=[pronounceable_data, unpronounceable_data])
        scoring = dict(right= 0, total = 0)
        for idx, row in test_data.iterrows():
            update_progress(scoring["total"] / test_data.shape[0])
            if(is_pronounceable(row["word"]) == row["is_pronounceable"]):
                # print("guessed correctly that " + row["word"] + " is " + ("not " if not row["is_pronounceable"] else "") + "pronounceable")
                scoring["right"] += 1
            # else:
            #     print("guessed incorrectly that " + row["word"] + " is " + ("" if not row["is_pronounceable"] else "not ") + "pronounceable")
            scoring["total"] += 1
        
        accuracy_dict["Heuristic Model"] = (scoring["right"] / scoring["total"])

### ML Test
Now that we have the ability to turn words into vectors (through one-hot encoding bigrams), we can begin to train models on these vectors. Since we are looking for a one-vs-one classification, we can use, for example:
1. Naive Bayes
2. K Nearest Neighbors
3. Semi-Supervised Learning
4. Support Vector Machines
5. Neural Network

### Training / Testing Data
Splits our dataset containing 3000 pronounceable and 3000 unpronouceable words into training and testing sets used to fit and evaluate our ML models.

In [7]:
from sklearn.model_selection import train_test_split
import numpy as np
size = 3000 # size of each half of the dataset
data_set = get_dataset(size)

X_vectors = list(get_vectors_for_series(data_set["word"], label=""))

# Convert from list of np arrays to single 2darray
X = np.array([x for x in X_vectors])
y = np.array([np.array(1 if i else 0) for i in data_set["is_pronounceable"]])
X_train, X_test, y_train, y_test = train_test_split(X, list(y), test_size=0.25, random_state=7)

Progress: [####################] 99.9%


### Naive Bayes

In [8]:
from sklearn.naive_bayes import BernoulliNB

naive_bayes_model = BernoulliNB()
naive_bayes_model.fit(X_train, y_train)

accuracy_dict["Naive Bayes"] = naive_bayes_model.score(X_test, y_test)

### K Nearest Neighbors

In [9]:
from sklearn.neighbors import KNeighborsClassifier

n_neighbors = 10
knn_model = KNeighborsClassifier(n_neighbors=n_neighbors).fit(X_train, y_train)
accuracy_dict[f"K-Nearest-Neighbors ({n_neighbors} neighbors)"] = knn_model.score(X_test, y_test)

### Semi-supervised Learning


In [10]:
from sklearn.semi_supervised import LabelPropagation

# get a "LOT" of extra data, and label it as -1

words_unlabeled = pd.Series(list(get_n_pronounceable_words(4500).union(get_n_unpronounceable_words(4500))))
X_unlabeled = np.array([i for i in list(get_vectors_for_series(words_unlabeled))])
y_unlabeled = np.array([-1 for _ in X_unlabeled])


X_mixed = np.concatenate((X_train, X_unlabeled), axis=0)

y_mixed = np.concatenate((y_train, y_unlabeled), axis=0)


semi_supervised_model = LabelPropagation().fit(X_mixed, y_mixed)


accuracy_dict["Semi Supervised (Label Propagation)"] = (semi_supervised_model.score(X_test, y_test))

Progress: [####################] 100.0%


### Support Vector Machine

In [11]:
from sklearn.svm import NuSVC

#TODO Tune parameters
svc_model = NuSVC().fit(X_train, y_train)
accuracy_dict["Support Vector Machine (NuSVC)"]= (svc_model.score(X_test, y_test))

### Neural Network

In [12]:
from sklearn.neural_network import MLPClassifier

#  TODO Tune parameters:
nn_model = MLPClassifier(learning_rate = "invscaling").fit(X_train, y_train)
accuracy_dict["Neural Network (Multi-Layer Perceptron)"] = (nn_model.score(X_test, y_test))

In [45]:
print("Stats for most recent run:")
for model in accuracy_dict:
    print(f"Accuracy for {model}: {str(accuracy_dict[model]*100)[:5]}%")

Stats for most recent run:
Accuracy for Heuristic Model: 88.7%


The accuracies of our models were as such:

Naive Bayes': 96.26%
KNN: 91.2%
Semi-Supervised: 87.83%
Support Vector Machine: 94.73%
Neural Network: 87.66%

Additionally the accuracy of our heuristic was 88.7%

## Task 4: Hypothesis Testing
In order to compare the effectiveness of our heuristic to that of our machine learning algorithms, we decided to run a hypothesis test comparing the outcome of our heuristic to that of our most effective algorithm, support vector machine.

H0: Our heuristic is as accurate as support vector machine
H1: Our heuristic is not as accurate as support vector machine

In order to test these hypotheses, we would run our naive bayes algorithm 30 times and get corresponding values for accuracy for each iteration. Applying the central limit theorem (because n ≥ 30), we then can approximate the distribution of the accuracy from the SVM algorithm by a normal distribution. We then want to test whether the mean of this distribution could potentially be equal to the accuracy derived from our heuristic. To do this we can perform a one sample t-test, seeing as we are only using one sample of accuracy values and our sample comes from a distribution which we can estimate as normal without knowing the population variance.

In [50]:
accuracies = []
# increase sample size by increasing n
best_model = NuSVC()
n = 30
for i in range(n):
    size = 1500 # size of each half of the dataset
    data_set = get_dataset(size)

    X_vectors = list(get_vectors_for_series(data_set["word"], label=f"Iteration {i+1}/{n}"))

    # Convert from list of np arrays to single 2darray
    X = np.array([x for x in X_vectors])
    y = np.array([np.array(1 if i else 0) for i in data_set["is_pronounceable"]])
    X_train, X_test, y_train, y_test = train_test_split(X, list(y), test_size=0.25, random_state=7)
    model = best_model.fit(X_train, y_train)
    accuracies.append(svc_model.score(X_test, y_test))
    


Iteration 30/30
Progress: [####################] 99.8%


In [54]:
from scipy import stats

assert len(accuracies) == 30
# all values in accuracies list are probabilities
assert max([0 if is_probability(sample) else 1 for sample in accuracies]) == 0
assert is_probability(accuracy_dict["Heuristic Model"])

one_sample = stats.ttest_1samp(accuracies, accuracy_dict["Heuristic Model"])
print("The t-statistic is %.3f and the p-value is %.3f." % one_sample)

The t-statistic is 66.922 and the p-value is 0.000.


After running the SVM algorithm 30 times and storing the accuracy data into a list, we can call the ttest_1samp function from the scipy package in order to perform this test, passing in the accuracy from our heuristic as the value for H0.


## Task 5: Insights attained
We reject the null hypothesis that our heuristic is as accurate as the support vector machine. However, our model still had a rather high mark for accuracy ass it predicted the probability that a word was pronounceable or not with an 88.7% accuracy. It is worth noting that while our algorithm did run slower than the support vector machine, itr was more memory efficient since we did not have to one-hot encode all of the bigrams. Even though we were unable to develop a model that had perfect predictions, it did help us all learn a little bit more about the English language and the way that syllables are used to form words, which was an invaluable bit of knowledge.
